# **Runnables**

Problem-solving typically involves a series of steps. When using LLMs, some applications have relatively complex requirements and a sequence of calls for performing an operation. 

Let's now learn how to put multiple chains together in an organized way.

## **What's covered?**
1. What is a Runnable?
2. RunnableLambda - Converting Custom Python Functions to Runnables
3. What are RunnableSequence?
4. Example: Putting Multiple Runnable in RunnableSequence
5. What are RunnableParallel?
6. What are RunnablePassthrough?
    - Just passing through
    - Adding new keys to input
7. Case Study: Building an AI Research Assistant

## **What is a Runnable?**
In LangChain, runnables are powerful abstractions representing any callable unit of work. They encapsulate and manage various tasks, including LLM calls, database queries and calls to external APIs.

All the components seen before i.e. Templates, Chat Models and Output Parsers inherit the RunnableInterface that enables them to work consistently. Methods like invoke(), stream() and batch() are common for all runnable components.

## **RunnableLambda - Converting Custom Python Functions to Runnables**

In [4]:
def sum_method(x: int) -> int:
    return x + x

def multiply_method(x: int) -> int:
    return x * x


print(type(sum_method))
print(type(multiply_method))

<class 'function'>
<class 'function'>


In [5]:
# Converting methods to Runnables
from langchain_core.runnables import RunnableLambda

runnable_1 = RunnableLambda(sum_method)
runnable_2 = RunnableLambda(multiply_method)

print(type(runnable_1))
print(type(runnable_2))

<class 'langchain_core.runnables.base.RunnableLambda'>
<class 'langchain_core.runnables.base.RunnableLambda'>


## **What are `RunnableSequence`?**

Note that all the chains are RunnableSequence.

LangChain implements the RunnableInterface, which allows the composition or chaining of various components into a RunnableSequence.  

You can compose these Runnable objects together to create a pipeline of operations.

**Syntax A:**  
```python
from langchain_core.runnables import RunnableSequence

chain = RunnableSequence(
    first=runnable_a, 
    middle=[runnable_b, runnable_c],
    last=runnable_d
)
```

**Syntax B:**  
```python
chain = runnable_a | runnable_b | runnable_c | runnable_d
```

In [11]:
# Defining a Runnable Sequence
from langchain_core.runnables import RunnableSequence

runnable_sequence = RunnableSequence(first=runnable_1, last=runnable_2)

runnable_sequence.invoke(2)

16

In [12]:
# Another way to define Runnable Sequence

runnable_sequence = runnable_1 | runnable_2

runnable_sequence.invoke(3)

36

In [13]:
# !pip install grandalf

In [14]:
runnable_sequence.get_graph().print_ascii()

   +------------------+    
   | sum_method_input |    
   +------------------+    
             *             
             *             
             *             
      +------------+       
      | sum_method |       
      +------------+       
             *             
             *             
             *             
    +-----------------+    
    | multiply_method |    
    +-----------------+    
             *             
             *             
             *             
+------------------------+ 
| multiply_method_output | 
+------------------------+ 


## **Example: Putting Multiple Runnable in RunnableSequence**

In [22]:
from datetime import datetime

from langchain_core.runnables import RunnableLambda, RunnableSequence


# Define the transformations as simple functions
def greet(name):
   return f"Hello, {name}!"


def append_datetime(text):
   current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
   return f"{text} The current date and time is {current_datetime}"


def to_uppercase(text):
   return text.upper()


def add_exclamation(text):
   return f"{text}!"


# Wrap the functions in RunnableWrapper
greet_runnable = RunnableLambda(lambda x: greet(x))
datetime_runnable = RunnableLambda(lambda x: append_datetime(x))
uppercase_runnable = RunnableLambda(lambda x: to_uppercase(x))
exclamation_runnable = RunnableLambda(lambda x: add_exclamation(x))


# Create a RunnableSequence with the wrapped runnables
chain = RunnableSequence(
   first=greet_runnable,
   middle=[datetime_runnable, uppercase_runnable],
   last=exclamation_runnable,
)

# Apply the chain to some input data
input_data = "Alice"
result = chain.invoke(input_data)
print(result)

HELLO, ALICE! THE CURRENT DATE AND TIME IS 2025-12-01 19:07:37!


In [23]:
# Create a RunnableSequence with the wrapped runnables
chain = greet_runnable | datetime_runnable | uppercase_runnable | exclamation_runnable

# Apply the chain to some input data
input_data = "Alice"
result = chain.invoke(input_data)
print(result)

HELLO, ALICE! THE CURRENT DATE AND TIME IS 2025-12-01 19:07:37!


## **What are `RunnableParallel`?**

Runnable that runs a mapping of Runnables in parallel, and returns a mapping of their outputs.

RunnableParallel is one of the two main composition primitives for the LCEL, alongside RunnableSequence. It invokes Runnables concurrently, providing the same input to each.

**Syntax:**
```python
from langchain_core.runnables import RunnableParallel

par = RunnableParallel({
    'key_a': runnable_a, 
    'key_b': runnable_b,
    ...
})
```

In [1]:
def sum_method(x: int) -> int:
    return x + x

def multiply_method(x: int) -> int:
    return x * x

In [2]:
# Converting methods to Runnables
from langchain_core.runnables import RunnableLambda

runnable_1 = RunnableLambda(sum_method)
runnable_2 = RunnableLambda(multiply_method)

In [7]:
# Defining a Runnable Parallel
from langchain_core.runnables import RunnableParallel

runnable_parallel = RunnableParallel({'sum_out': runnable_1, 'mult_out': runnable_2})

# # Another Way to Define RunnableParallel
# runnable_parallel = RunnableParallel(sum_out=runnable_1, mult_out=runnable_2)

runnable_parallel.invoke(5)

{'sum_out': 10, 'mult_out': 25}

In [8]:
runnable_parallel.get_graph().print_ascii()

     +---------------------------------+         
     | Parallel<sum_out,mult_out>Input |         
     +---------------------------------+         
              ***            ***                 
            **                  **               
          **                      **             
+------------+              +-----------------+  
| sum_method |              | multiply_method |  
+------------+              +-----------------+  
              ***            ***                 
                 **        **                    
                   **    **                      
    +----------------------------------+         
    | Parallel<sum_out,mult_out>Output |         
    +----------------------------------+         


## **RunnablePassThrough**

Imagine you have a message, and you want to hand that exact message over to the next person in line, without changing it at all. That's RunnablePassthrough! It simply takes whatever input it receives and passes it along, untouched, to the next runnable in your chain.

On its own, it might seem pointless. **"Why would I add a component that does nothing?"** you might ask.
> - The magic isn't in what it does by itself, but in how it enables other, more complex operations (especially when combined with .assign() or parallel processing, which we'll see next!).
> - Think of it as a placeholder or a distributor of the original input.


Runnable to passthrough inputs either:

- unchanged - RunnablePassthrought()  
(or)
- with additional keys - with the help of assign method - RunnablePassthrough.assign(variable_name=value)

This Runnable behaves almost like the identity function, except that it can be configured to add additional keys to the output, if the input is a dict.


### **Just passing through**

In [11]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

def inspect_input(input_data):
    print(f"Data Recieved from RunnableLambda: {input_data}")
    return input_data # Always return something for the chain to continue

inspector = RunnableLambda(inspect_input)

chain = RunnablePassthrough() | inspector

In [12]:
result = chain.invoke("Hello LangChain!")
print(result)

Data Recieved from RunnableLambda: Hello LangChain!
Hello LangChain!


In [13]:
result = chain.invoke({"name": "Alice", "age": 30})
print(result)

Data Recieved from RunnableLambda: {'name': 'Alice', 'age': 30}
{'name': 'Alice', 'age': 30}


In [14]:
from langchain_core.runnables import (
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

runnable = RunnableParallel({
    "origin":RunnablePassthrough(),
    "modified":lambda x: x+1
})

runnable.invoke(1)

{'origin': 1, 'modified': 2}

In [15]:
runnable.get_graph().print_ascii()

  +--------------------------------+  
  | Parallel<origin,modified>Input |  
  +--------------------------------+  
            ***         **            
           *              **          
         **                 *         
+-------------+          +--------+   
| Passthrough |          | Lambda |   
+-------------+          +--------+   
            ***         **            
               *      **              
                **   *                
 +---------------------------------+  
 | Parallel<origin,modified>Output |  
 +---------------------------------+  


In [16]:
# Modified Example

runnable = {"original": RunnablePassthrough()} | RunnableLambda(lambda x: x["original"]+1)

# # Guess the output of the following
# runnable.invoke(1)

In [17]:
runnable.get_graph().print_ascii()

+-------------------------+  
| Parallel<original>Input |  
+-------------------------+  
              *              
              *              
              *              
      +-------------+        
      | Passthrough |        
      +-------------+        
              *              
              *              
              *              
        +--------+           
        | Lambda |           
        +--------+           
              *              
              *              
              *              
      +--------------+       
      | LambdaOutput |       
      +--------------+       


### **Adding New Keys to Input**

RunnablePassthrough.assign() takes your existing input, runs some other runnable(s) to create new keys in the input dictionary, and then passes this expanded dictionary to the next step.

Imagine you have a file folder with some initial documents. You want to add new documents to that same folder, based on what's already inside, and then pass the entire, updated folder to the next person. assign() is like adding those new documents.

Syntax:
```python
some_chain = RunnablePassthrough.assign(
    new_key_1 = some_runnable_that_produces_value_for_new_key_1,
    new_key_2 = some_runnable_that_produces_value_for_new_key_2,
    # ... and so on
)
```

**Note:**
1. The `some_runnable_that_produces_value_for_new_key_X` will receive the original input that was passed to some_chain.
2. The output of that runnable becomes the value for `new_key_X`.
3. The output of assign() itself will be the original input dictionary merged with these newly added key-value pairs.

**Important:**  
In some cases, it may be useful to pass the input through while adding some keys to the output. In this case, you can use the assign method:

In [19]:
from langchain_core.runnables import RunnablePassthrough
import random

# Fake LLM for the example
def fake_llm(prompt: str) -> str:
    word_list = ["python", "data", "science", "genai", "developer"]
    return random.choice(word_list)

runnable = {
    'llm1':  fake_llm,   # Passing the Function as a callback
    'llm2':  fake_llm,   # Passing the Function as a callback
} | RunnablePassthrough.assign(
    total_chars=lambda inputs: len(inputs['llm1'] + inputs['llm2'])
)

runnable.invoke('hello')

{'llm1': 'genai', 'llm2': 'developer', 'total_chars': 14}

In [20]:
runnable.get_graph().print_ascii()

   +--------------------------+      
   | Parallel<llm1,llm2>Input |      
   +--------------------------+      
           **         **             
         **             **           
        *                 *          
+----------+          +----------+   
| fake_llm |          | fake_llm |   
+----------+          +----------+   
           **         **             
             **     **               
               *   *                 
  +---------------------------+      
  | Parallel<llm1,llm2>Output |      
  +---------------------------+      
                 *                   
                 *                   
                 *                   
  +----------------------------+     
  | Parallel<total_chars>Input |     
  +----------------------------+     
           **         **             
         **             **           
        *                 *          
 +--------+          +-------------+ 
 | Lambda |          | Passthrough | 
 +--------+ 

## **Case Study: Building an AI Research Assistant**

In [26]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [27]:
# Setup API Key

f = open('keys/.openai_api_key.txt')

OPENAI_API_KEY = f.read()

In [40]:
output_parser = StrOutputParser()

chat_model = ChatOpenAI(api_key=OPENAI_API_KEY,
                               model="gpt-4o-mini",
                               temperature=0.0)

In [41]:
writer_chat_template = ChatPromptTemplate.from_messages([
    ("system", """You are a research assistant and scientific writer.
                  You take in requests about the topics and write organized research reports on those topics.
                  Also you share the appropriate references at the end of report."""), 
    ("human", """Write an organized research report about {topic}.""")
])

writer_chain = writer_chat_template | chat_model | output_parser

# research_report = writer_chain.invoke({"topic": "how transformers algorithm works?"})

# print(research_report)

In [42]:
reviewer_chat_template = ChatPromptTemplate.from_messages([
    ("system", """You are a reviewer for research reports. 
                  You take in research reports and provide a feedback on them."""
    ), 
    ("human", """Provide feedback as 5 concise bullet points on this research report: 
                 
                 {report}"""
    )
])

reviewer_chain = reviewer_chat_template | chat_model | output_parser

# report_feedback = reviewer_chain.invoke({"report": research_report})

# print(report_feedback)

In [43]:
final_writer_chat_template = ChatPromptTemplate.from_messages([
    ("system", """You are a research assistant and scientific writer.
                  You take in a research report in a set of bullet points with feedback to improve.
                  You revise the research report based on the feedback and write a final version."""
    ), 
    ("human", """Write a reviewed and improved version of research report: 

                {report}
                
                based on this feedback:
                
                {feedback}"""
    )
])

final_writer_chain = final_writer_chat_template | chat_model | output_parser

# final_report = final_writer_chain.invoke({"report": research_report, "feedback": report_feedback})

# print(final_report)

### **What are these chains?**

Given that we have created three chains above, let's now analyse what these chains are?

In [44]:
print(type(writer_chain))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [45]:
print(type(reviewer_chain))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [46]:
print(type(final_writer_chain))

<class 'langchain_core.runnables.base.RunnableSequence'>


### **Composing the Chains together**

**RunnablePassthrough** for passing data unchanged from previous steps for use as input in later steps.

<img src="images/composing_chains.jpg">

In [29]:
from langchain_core.runnables import RunnablePassthrough

composed_chain = {"report" : writer_chain} | RunnablePassthrough().assign(feedback=reviewer_chain) | final_writer_chain

final_report = composed_chain.invoke({"topic": "What are Runnables in LangChain?"})

In [37]:
# # Another Way

# report_output = RunnableLambda(lambda out : {"report" : out})

# composed_chain = writer_chain | report_output | RunnablePassthrough().assign(feedback=reviewer_chain) | final_writer_chain
# final_report = composed_chain.invoke({"topic": "What are Runnables in LangChain?"})

In [36]:
from IPython.display import Markdown

Markdown(final_report)

# Research Report: Runnables in LangChain

## Introduction
LangChain is an innovative framework designed to facilitate the development of applications that utilize language models. A key component of LangChain is the concept of "Runnables," which serve as fundamental building blocks within the framework. Runnables enable developers to create modular and reusable components that can execute various tasks related to language processing. This report provides a comprehensive overview of Runnables in LangChain, including their definition, functionality, use cases, advantages, and future directions.

## Definition of Runnables
Runnables in LangChain are abstractions that encapsulate specific tasks or operations that can be executed independently or in conjunction with other Runnables. Each Runnable can take inputs, perform computations or transformations, and produce outputs. This modular approach allows developers to compose complex workflows by chaining multiple Runnables together.

**Key Takeaway**: Runnables are self-contained units of work that enhance modularity and reusability in application development.

## Functionality of Runnables
Runnables in LangChain are designed to be flexible and versatile, representing a wide range of operations, including:

1. **Text Generation**: Runnables can invoke language models to generate text based on given prompts.
2. **Data Transformation**: They can process and transform data, such as cleaning or formatting text.
3. **API Calls**: Runnables can make API requests to external services, integrating additional functionalities into the application.
4. **Conditional Logic**: Developers can implement conditional logic within Runnables to control the flow of execution based on specific criteria.

### Structure of Runnables
A typical Runnable in LangChain consists of:
- **Input Parameters**: The data or parameters required for execution.
- **Execution Logic**: The core functionality that defines what the Runnable does.
- **Output**: The result produced after execution, which can be passed to other Runnables or returned to the user.

**Key Takeaway**: The structure of Runnables promotes clarity and organization in code, making it easier for developers to manage complex tasks.

## Use Cases
Runnables can be employed in various applications, including:

1. **Chatbots**: Runnables can handle user queries, generate responses, and manage conversation flows.
   - *Example*: A chatbot that uses Runnables to process user input, generate a response using a language model, and log the conversation for future analysis.
   
2. **Content Creation**: They can automate the generation of articles, summaries, or other written content based on user-defined parameters.
   - *Example*: A content generation tool that utilizes Runnables to create blog posts based on keywords provided by the user.

3. **Data Analysis**: Runnables can process and analyze text data, extracting insights or performing sentiment analysis.
   - *Example*: A sentiment analysis application that uses Runnables to analyze customer feedback and categorize sentiments.

4. **Workflow Automation**: By chaining multiple Runnables, developers can create complex workflows that automate repetitive tasks.
   - *Example*: An automated reporting system that collects data, generates reports, and sends them via email using a series of interconnected Runnables.

**Key Takeaway**: Runnables are versatile and can be applied across various domains, enhancing the functionality of applications.

## Advantages of Runnables
The use of Runnables in LangChain offers several advantages:

1. **Modularity**: Runnables promote a modular design, allowing developers to build and maintain applications more easily.
2. **Reusability**: Once created, Runnables can be reused across different projects, saving time and effort.
3. **Scalability**: The ability to chain Runnables together enables the development of scalable applications that can handle complex tasks.
4. **Ease of Integration**: Runnables can easily integrate with other components of LangChain and external services, enhancing the overall functionality of applications.

**Key Takeaway**: Runnables enhance development efficiency and application performance through their modular and reusable nature.

## Comparative Analysis
When compared to similar concepts in other frameworks, Runnables in LangChain stand out due to their flexibility and ease of integration. For instance, while other frameworks may offer similar modular components, Runnables provide a more intuitive interface for chaining operations and managing dependencies. This unique approach allows developers to create complex workflows with minimal overhead.

## Future Directions
As the LangChain framework evolves, there are several potential developments for Runnables:

1. **Enhanced Error Handling**: Future versions could introduce more robust error handling mechanisms to improve the reliability of Runnables.
2. **Performance Optimization**: Ongoing efforts to optimize the execution speed of Runnables could enhance application performance.
3. **Expanded Library of Runnables**: The introduction of a broader library of pre-built Runnables could facilitate quicker development for common tasks.
4. **Community Contributions**: Encouraging community contributions could lead to innovative use cases and enhancements for Runnables.

**Key Takeaway**: The future of Runnables in LangChain is promising, with opportunities for growth and improvement that can further enhance their utility.

## Conclusion
Runnables are a core feature of the LangChain framework, providing a powerful and flexible way to build language model applications. By encapsulating tasks into modular components, Runnables facilitate the development of complex workflows while promoting reusability and scalability. As the demand for language processing applications continues to grow, understanding and leveraging Runnables will be essential for developers working within the LangChain ecosystem.

## References
1. LangChain Documentation. (2023). Retrieved from [LangChain Documentation](https://langchain.readthedocs.io/en/latest/)
2. LangChain GitHub Repository. (2023). Retrieved from [LangChain GitHub](https://github.com/hwchase17/langchain)
3. OpenAI. (2023). Language Models: A Survey. Retrieved from [OpenAI Research](https://openai.com/research/language-models)